## Script to move raw MACS files based on extracted footprints files 
**author:** Ingmar Nitze, Tabea Rettelbach, Simon Schäffler

**contact:** ingmar.nitze@awi.de

**version date:** 2022-02-23

**repository and other tools** https://github.com/awi-response/MACS_tools

## Contents
1. Setup folder structure
2. convert MACS files to TIFF with *mipps* including devignetting
3. Rescale image values **Optional**
4. Crop corners of images **Optional**
5. Prepare nav files for Pix4d

## Settings 

* prefer full/absolute paths
* Create processing template automatically

In [1]:
# Set project directory here, where you want to process your data
#PROJECT_DIR = r''
PROJECT_DIR = r'D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_CODE' # SET Project output

# Set raw data dir here for the speicific image acquisition project
#path_infiles = r''
path_infiles = r'N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190727-235440_15L_Ketik_fire_flight_plan_v3'

# determine which sensors to include in processing (possible options: 'left', 'right', 'nir')
sensors = ['left', 'right', 'nir']

# Set CROP CORNER if 
CROP_CORNER = 0 # SET to 1 if you want to crop corners (set to NoData)
DISK_SIZE = 5200 # Cropping diameter, the larger the fewer no data

# SET SCALING 
SCALING = 1
SCALE_LOW = True # Set to True to use calculated lower boundary
SCALE_HIGH = True # Set to True to use calculated upper boundary

### Imports 

In [2]:
import geopandas as gpd
import shutil
import os
import glob
import pandas as pd
from IPython.display import clear_output
import sys
import numpy as np
import tqdm
import zipfile
from pathlib import Path
from joblib import delayed, Parallel, wrap_non_picklable_objects
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from processing_utils import *

In [3]:
pwd = Path(os.getcwd())

#### Fixed Settings
* These Settings can be kept

In [4]:
CODE_DIR = pwd
MIPPS_DIR = r'C:\Program Files\DLR MACS-Box\bin'
MIPPS_BIN = r'..\tools\MACS\mipps.exe'
EXIF_PATH = Path(CODE_DIR / Path(r'exiftool\exiftool.exe'))
mipps_script_dir = Path('mipps_scripts')

In [5]:
mipps_script_nir = '33552_all_taps_2018-09-26_12-58-15_modelbased.mipps'
mipps_script_right = '33576_all_taps_2018-09-26_13-13-43_modelbased.mipps'
mipps_script_left = '33577_all_taps_2018-09-26_13-21-24_modelbased.mipps'

mipps_script_nir = pwd / mipps_script_dir / mipps_script_nir
mipps_script_right = pwd / mipps_script_dir / mipps_script_right
mipps_script_left = pwd / mipps_script_dir / mipps_script_left

In [6]:
DATA_DIR = Path(PROJECT_DIR) / '01_rawdata' / 'tif'
OUTDIR = {'right': DATA_DIR / Path('33576_Right'),
          'left':DATA_DIR / Path('33577_Left'),
          'nir':DATA_DIR / Path('33552_NIR')}
tag = {'right':'MACS_RGB_Right_33576',
       'left':'MACS_RGB_Left_33577',
       'nir':'MACS_NIR_33552'}

In [7]:
# Path of filtered footprints file (.shp file)
path_footprints = os.path.join(PROJECT_DIR, '02_studysites','footprints.shp')
outdir = os.path.join(PROJECT_DIR, '01_rawdata','tif')

#### Prepare processing dir 
* check if exists

In [8]:
zippath = os.path.join(CODE_DIR, 'processing_folder_structure_template.zip')
nav_script_path = os.path.join(CODE_DIR, 'pix4dnav.py')

In [9]:
with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(PROJECT_DIR)
shutil.copy(nav_script_path, outdir)

'D:\\pix4d_Processing\\ThawTrendAir_2019\\Image_Test_CODE\\01_rawdata\\tif\\pix4dnav.py'

### Manual Selection of footprints 

Now select footprints and export selection as ***footprints.shp*** to ***02_footprints*** in your working directory

#### Load filtered footprints file 

In [10]:
#df_final = prepare_df_for_mipps(path_footprints, path_infiles)
df_final = prepare_df_for_mipps(path_footprints, path_infiles)

#### Workaround to deal with spaces in path" 

In [11]:
df_final['full_path'] = df_final.apply(lambda x: f'"{x.full_path}"', axis=1)

In [12]:
print("Total number of images:", len(df_final))
print("NIR images:", (df_final['Looking'] == 'center').sum())
print("RGB right images:", (df_final['Looking'] == 'right').sum())
print("RGB left images:", (df_final['Looking'] == 'left').sum())

Total number of images: 499
NIR images: 206
RGB right images: 182
RGB left images: 111


#### Run Process 

In [13]:
os.chdir(MIPPS_DIR)

In [14]:
max_roll = 3 # Select maximum roll angle to avoid image issues - SET in main settings part?
chunksize = 20 # this is a mipps-script thing

In [15]:
# this is relevant for NIR only
if 'nir' in sensors:
    looking = 'center'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_nir = df_final[q]
    print(len(df_nir))
    for df in tqdm.tqdm_notebook(np.array_split(df_nir, len(df_nir) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_nir} -o={outdir} -j=4 {outlist}'
        os.system(s)
        #print(s)

201


C:\Users\initze\AppData\Local\Temp\ipykernel_35312\1512507586.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for df in tqdm.tqdm_notebook(np.array_split(df_nir, len(df_nir) // chunksize)):


  0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
# this is RGB
if 'right' in sensors:
    looking = 'right'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_right = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_right, len(df_right) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_right} -o={outdir} -j=4 {outlist}'
        os.system(s)

C:\Users\initze\AppData\Local\Temp\ipykernel_35312\2076830559.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for df in tqdm.tqdm_notebook(np.array_split(df_right, len(df_right) // chunksize)):


  0%|          | 0/9 [00:00<?, ?it/s]

In [17]:
if 'left' in sensors:
    looking = 'left'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_left = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_left, len(df_left) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_left} -o={outdir} -j=4 {outlist}'
        os.system(s)

C:\Users\initze\AppData\Local\Temp\ipykernel_35312\1335427181.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for df in tqdm.tqdm_notebook(np.array_split(df_left, len(df_left) // chunksize)):


  0%|          | 0/5 [00:00<?, ?it/s]

### Rescale image values 

#### Image Statistics 

In [18]:
if SCALING:
    %time df_stats = get_image_stats_multi(OUTDIR, sensors, nth_images=10, quiet=True)
    #absolute
    if SCALE_LOW:
        scale_lower = int(df_stats['min'].mean().round())
    else:
        scale_lower = 1
    if SCALE_HIGH:
        scale_upper = int(df_stats['max'].mean().round())
    else:
        scale_upper = 2*16-1
    print(f'Mean of minimums: {scale_lower}')
    print(f'Mean of maximums: {scale_upper}')

CPU times: total: 250 ms
Wall time: 9.98 s
Mean of minimums: 2548
Mean of maximums: 29623


#### Run scaling
* minimum default to 1
* consistency for final index calculation

In [19]:
if SCALING:
    n_jobs = 50
    for sensor in sensors:
        print(f'Processing {sensor}')
        #shutter_factor
        images = list(OUTDIR[sensor].glob('*.tif'))[:]
        if sensor in ['right', 'left']:
            shutter_factor = get_shutter_factor(OUTDIR, sensors)
            print(f'RGB to NIR factor = {shutter_factor}')
        else:
            shutter_factor = 1
        
        %time _ = Parallel(n_jobs=n_jobs)(delayed(write_new_values)(image, scale_lower, scale_upper, shutter_factor=shutter_factor, tag=True) for image in tqdm.tqdm_notebook(images[:]))

Processing left
RGB to NIR factor = 1.0


<timed exec>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/106 [00:00<?, ?it/s]

CPU times: total: 391 ms
Wall time: 30.4 s
Processing right
RGB to NIR factor = 1.0


  0%|          | 0/180 [00:00<?, ?it/s]

CPU times: total: 188 ms
Wall time: 39.4 s
Processing nir


  0%|          | 0/201 [00:00<?, ?it/s]

CPU times: total: 219 ms
Wall time: 11.8 s


#### Crop Corners of images 

In [20]:
if CROP_CORNER:
    #mask = make_mask((3232, 4864), disksize=DISK_SIZE)
    for sensor in sensors[:]:
        mask = make_mask((3232, 4864), disksize=DISK_SIZE)
        images = list(OUTDIR[sensor].glob('*'))
        if sensor != 'nir':
            mask = np.r_[[mask]*3]
        #%time _ = [mask_and_tag(image, mask, tag=None) for image in tqdm.tqdm_notebook(images)]
        %time _ = Parallel(n_jobs=4)(delayed(mask_and_tag)(image, mask, tag=None) for image in tqdm.tqdm_notebook(images))

#### Write exif information into all images 

In [21]:
for sensor in tqdm.tqdm_notebook(sensors):
    print(sensor)
    %time write_exif(OUTDIR[sensor], tag[sensor], EXIF_PATH)

C:\Users\initze\AppData\Local\Temp\ipykernel_35312\2113234875.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sensor in tqdm.tqdm_notebook(sensors):


  0%|          | 0/3 [00:00<?, ?it/s]

left
D:\pix4d_Processing\MACS_tools\exiftool\exiftool.exe -overwrite_original -Model="MACS_RGB_Left_33577" D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_CODE\01_rawdata\tif\33577_Left
CPU times: total: 0 ns
Wall time: 18.8 s
right
D:\pix4d_Processing\MACS_tools\exiftool\exiftool.exe -overwrite_original -Model="MACS_RGB_Right_33576" D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_CODE\01_rawdata\tif\33576_Right
CPU times: total: 0 ns
Wall time: 31.3 s
nir
D:\pix4d_Processing\MACS_tools\exiftool\exiftool.exe -overwrite_original -Model="MACS_NIR_33552" D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_CODE\01_rawdata\tif\33552_NIR
CPU times: total: 0 ns
Wall time: 13.5 s


#### Nav

In [22]:
navfile = list(Path(path_infiles).glob('*nav.txt'))[0]

In [23]:
shutil.copy(navfile, OUTDIR['nir'].parent / 'nav.txt')

WindowsPath('D:/pix4d_Processing/ThawTrendAir_2019/Image_Test_CODE/01_rawdata/tif/nav.txt')

In [24]:
os.chdir(OUTDIR['nir'].parent)
os.system('python pix4dnav.py')

0

### Run test to calculate and compare stats of every n-th image
* reduce calculation
* simplify

#### Show statistics of subsets 

In [ ]:
print('Absolute Minimum')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print (df_tmp['min'].min())
print('Absolute Mean')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['mean'].mean())
print('Absolute Max')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['max'].max())
print('Mean Max')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['max'].mean())
print('Minimum perc01')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['p01'].min())
print('maximum perc99')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['p99'].max())

In [ ]:
%time df_01 = get_image_stats_multi(OUTDIR, sensors, nth_images=1, quiet=True)

In [ ]:
%time df_05 = get_image_stats_multi(OUTDIR, sensors, nth_images=5, quiet=True)

In [ ]:
%time df_20 = get_image_stats_multi(OUTDIR, sensors, nth_images=20, quiet=True)

In [ ]:
#RGB
scale_upper = df_20.query('sensor in ("left", "right")')['max'].mean().round()
scale_lower = df_20.query('sensor in ("left", "right")')['min'].mean().round()
print(scale_lower, scale_upper)

In [ ]:
#NIR
scale_upper = df_20.query('sensor in ("nir")')['max'].mean().round()
scale_lower = df_20.query('sensor in ("nir")')['min'].mean().round()
print(scale_lower, scale_upper)

In [ ]:
# SCALING functions
def rescale(array, minimum, maximum, dtype=np.uint16, gain=1.):
    x = [0, 2**16-1]
    y = [minimum, maximum]
    slope, intercept, r_value, p_value, std_err = linregress(y,x)
    #print(slope,intercept)
    D = (array * gain) *slope + intercept
    D_round = np.around(np.clip(D, 1, 2**16-1))
    return np.array(D_round, np.uint16)
    
def write_new_values(image, minimum, maximum, shutter_factor=1, tag=True):
    with rasterio.open(image, mode='r+')as src:
        data = src.read()
        datanew = rescale(data, minimum, maximum, gain=shutter_factor)
        src.write(datanew)
        if tag:
            src.update_tags(VALUE_STRETCH_MINIMUM=minimum, VALUE_STRETCH_MAXIMUM=maximum)